# Train pix2pixHD to transform NAIP pre-flood image to xBD post-flooding image

### Experiment setup
- pre-flooding image and xBD post-flooding image
- bigger pix2pixHD
- binary masks 1024
- train on Train + Hold
- pip install tensorflow

### Display config

In [1]:
%%bash
cd ../..
# . scripts/config/flood.sh
. scripts/config/houston_west.sh
echo 'bucket_path: '${bucket_path}
echo 'seg_model_path: '${seg_model_path} 
echo 'seg_model_name: '${seg_model_name} 

echo 'root_dir: '${root_dir} 
echo 'dataroot: '${dataroot}
echo 'model_name: '${model_name}
echo 'ckpt_dir: '${ckpt_dir}
echo 'temp_dir: '${temp_dir}
echo 'results_dir: '${results_dir}

bucket_path: eie_floods/naip/houston_west/
seg_model_path: pretrained/pix2pix_seg/flood/scratch_1024_plus_houston_west5/
seg_model_name: latest_net_G.pth
root_dir: /home/jupyter/eie_vision/
dataroot: data/eie_floods/naip/houston_west/
model_name: conditional_binary/
ckpt_dir: temp/checkpoint/Pix2pixHD/
temp_dir: temp/Pix2pixHD/
results_dir: results/Pix2pixHD/


### Load data into instance

In [ ]:
%%bash
# set -xe
# Load data into instance (in terminal)
cd ../..; . scripts/config/flood.sh

# split_src=('train_A/' 'train_B/' 'train_mask/' 'test_A/' 'test_B/' 'test_mask/' 'hold_A/' 'hold_B/' 'hold_mask/')
# split_dst=('train_A/' 'train_B/' 'train_mask/' 'test_A/' 'test_B/' 'test_mask/' 'hold_A/' 'hold_B/' 'hold_mask/') 
split_src=('train_A/' 'test_A/' 'hold_A/')
split_dst=('train_A/' 'test_A/' 'hold_A/') 

# Combine train and hold
# split_src=('Train_A/' 'Train_B/' 'Train_mask/' 'Test_A/' 'Test_B/' 'Test_mask/' 'Hold_A/' 'Hold_B/' 'Hold_mask/')
# split_dst=('train_A/' 'train_B/' 'train_mask/' 'test_A/' 'test_B/' 'test_mask/' 'train_A/' 'train_B/' 'train_mask/') 

for ((i=0;i<${#split_src[@]};++i)); do
    mkdir -p ${root_dir}data/${bucket_path}${split_dst[i]}
    gsutil -m cp -r gs://${bucket_path}${split_src[i]}* ${root_dir}data/${bucket_path}${split_dst[i]}
    # Delete unused files
    rm -r ${root_dir}data/${bucket_path}${split_dst[i]}palu* | true
    #rm -r ${root_dir}data/${bucket_path}${split_dst[i]}midwest*
    rm -r ${root_dir}data/${bucket_path}${split_dst[i]}*matthew* | true
done

In [43]:
%%bash
cd ../..; . scripts/config/flood.sh
echo ${root_dir}${dataroot}
ls ${root_dir}${dataroot}/*_A/* | wc -l
ls ${root_dir}${dataroot}/*_B/* | wc -l
ls ${root_dir}${dataroot}/*_mask/* | wc -l

/home/jupyter/eie_vision/data/eie_floods/naip/houston_west/
5602
5602
5602


### (opt1) Generate segmentation masks for training imagery (in terminal)

In [10]:
!cd ../..; . scripts/config/flood.sh ;\
gsutil -m cp -r gs://eie_floods/${seg_model_path}${seg_model_name} ${root_dir}${seg_model_path}${seg_model_name}
# eie_floods/pretrained/pix2pix_seg/houston_west

Copying gs://eie_floods/pretrained/pix2pix_seg/houston_west/latest_net_G.pth...
| [1/1 files][207.6 MiB/207.6 MiB] 100% Done                                    
Operation completed over 1 objects/207.6 MiB.                                    


In [13]:
%%bash
cd ../..; . scripts/config/flood.sh

src_path=${dataroot}'test_B'
dst_path=${dataroot}'test_mask'

# Delete pre-existing segmentation masks
cd ${root_dir}
rm -r ${dst_path}

# Create segmentation masks
python src/models/Pix2pix-CycleGAN/test.py --phase to_segment \
--dataroot ${src_path} --direction AtoB \
--results_dir ${dst_path} \
--dataset_mode single \
--model pix2pix \
--checkpoints_dir ${seg_model_path} \
--name . --num_test 6000 --no_flip \
--gpu_ids 0 \
--max_dataset_size 6000 \
--batch_size 1  --load_size 1024 --crop_size 1024 
# --num_test 775
# TODO: eliminate FileNotFoundError: of .../test_mask./loses.csv'


# Rename files into original file names
cd ${dst_path}
mv to_segment_latest/images/*_fake_B.png .
rm -r to_segment_latest
# src_file='*${event_name}*_*[0-9]*_*${im_suffix}*.png'
src_file='*'${event_name}'*[0-9]*'${im_suffix}'*.png'
rename 's/_fake_B.png/.png/' ${src_file}

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: pretrained/pix2pix_seg/houston_west/	[default: ./checkpoints]
                crop_size: 256                           
                 dataroot: data/eie_floods/naip/houston_west/test_B	[default: None]
             dataset_mode: unaligned                     	[default: aligned]
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
               isFineTune: False                         	[default: No

rm: cannot remove 'data/eie_floods/naip/houston_west/test_mask': No such file or directory
Traceback (most recent call last):
  File "src/models/Pix2pix-CycleGAN/test.py", line 47, in <module>
    dataset = create_dataset(opt)  # create a dataset given opt.dataset_mode and other options
  File "/home/jupyter/eie_vision/src/models/Pix2pix-CycleGAN/data/__init__.py", line 57, in create_dataset
    data_loader = CustomDatasetDataLoader(opt)
  File "/home/jupyter/eie_vision/src/models/Pix2pix-CycleGAN/data/__init__.py", line 73, in __init__
    self.dataset = dataset_class(opt)
  File "/home/jupyter/eie_vision/src/models/Pix2pix-CycleGAN/data/unaligned_dataset.py", line 29, in __init__
    self.A_paths = sorted(make_dataset(self.dir_A, opt.max_dataset_size))   # load images from '/path/to/data/trainA'
  File "/home/jupyter/eie_vision/src/models/Pix2pix-CycleGAN/data/image_folder.py", line 26, in make_dataset
    assert os.path.isdir(dir), '%s is not a valid directory' % dir
AssertionError:

### (opt2 - recommended) Copy segmentation masks from non-adapted xBD dataset

In [ ]:
# Get segmentation masks (opt)
root_dir='/home/jupyter/eie_vision/'
src_path='data/xBD/'
dst_path='data/eie_floods/naip/xbd/images/'
cp -r ${root_dir}${src_path}train_mask ${root_dir}${dst_path}
cp -r ${root_dir}${src_path}val_mask/* ${root_dir}${dst_path}train_mask/
cp -r ${root_dir}${src_path}test_mask ${root_dir}${dst_path}
cd ${root_dir}${dst_path}train_mask

# Delete matthew
rm -r ${root_dir}data/eie_floods/naip/xbd/images/train_mask/*matthew*
rm -r ${root_dir}data/eie_floods/naip/xbd/images/*_mask/*matthew*

# Rename files into original file names
src_file='*hurricane-*_*[0-9]*_*_disaster*.png'
rename 's/_fake_B.png/.png/' ${src_file}
rename 's/training-set_train_images_hurricane/hurricane/' ${src_file}
rename 's/holdout-set_hold_images_labels_targets_hold_images_hurricane/hurricane/' ${src_file}
rename 's/test-set_test_images_labels_targets_test_images_hurricane/hurricane/' ${src_file}

### Train pix2pixHD

In [45]:
# Remove model and jupyter checkpoints
!cd /home/jupyter/eie_vision; . scripts/config/houston_west.sh ;\
rm -r ${root_dir}${ckpt_dir}${model_name} ;\
rm -r ${root_dir}${dataroot}*/.ipynb_checkpoints

rm: cannot remove '/home/jupyter/eie_vision/temp/checkpoint/Pix2pixHD/conditional_binary_houston_west/': No such file or directory
rm: cannot remove '/home/jupyter/eie_vision/data/eie_floods/naip/houston_west/*/.ipynb_checkpoints': No such file or directory


In [50]:
!cd /home/jupyter/eie_vision; . scripts/config/houston_west.sh ;\
python src/models/Pix2pixHD/train.py \
--no_lpips_loss --save_epoch_freq 15 --niter 100 \
--niter_decay 100 --checkpoints_dir ${ckpt_dir} --name ${model_name} \
--dataroot ${dataroot} --dataset_mode physics_aligned_bin --no_instance \
--label_nc 0 --input_nc 4 --batchSize 1 --gpu_ids 0 # --batchSize 8 --gpu_ids 0,1,2,3,4,5,6,7 \
# --tf_log
# --continue_train 

------------ Options -------------
augmentation_aggressive: False
batchSize: 1
beta1: 0.5
checkpoints_dir: temp/checkpoint/Pix2pixHD/
continue_train: False
data_type: 32
dataroot: data/eie_floods/naip/houston_west/
dataset_mode: physics_aligned_bin
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 4
instance_feat: False
isTrain: True
l1_pre: False
l1_pre_init_prob: 0.9
l1_pre_l1_loss_decay_epoch: 10.0
l1_pre_prob_decay: 0.005
label_feat: False
label_nc: 0
lambda_feat: 10.0
lambda_l1: 3.0
lambda_l1_decay: 0.03
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 8
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: conditional_binary_houston_west/
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_h

### Generate predictions over test set

In [11]:
%%bash
# Display number of test images
cd /home/jupyter/eie_vision; . scripts/config/flood.sh

ls ${dataroot}test_A | wc -l

394


In [12]:
!cd /home/jupyter/eie_vision; . scripts/config/flood.sh ;\
python src/models/Pix2pixHD/test.py \
--checkpoints_dir ${ckpt_dir} --name ${model_name} \
--phase test --how_many 2000 \
--dataroot ${dataroot} --dataset_mode physics_aligned_bin --no_instance \
--label_nc 0 --input_nc 4 --batchSize 8 --gpu_ids 0,1,2,3,4,5,6,7 \
--results_dir ${temp_dir}

------------ Options -------------
aspect_ratio: 1.0
augmentation_aggressive: False
batchSize: 8
checkpoints_dir: temp/checkpoint/Pix2pixHD/
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: data/eie_floods/naip/xbd/images/
dataset_mode: physics_aligned_bin
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0, 1, 2, 3, 4, 5, 6, 7]
how_many: 2000
input_nc: 4
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 8
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: conditional_binary/
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
normD: spectral
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: scale_width
results_dir: temp/Pix2pixHD/
serial_batches: False
slosh_inference: False
tf_log:

### Create segmentation masks for generated imagery and move imagery to results/ dir
We use the ./temp directly to work with the imagery and then carefully select imagery for the ./results directory, which is synced with the buckets.

In [13]:
%%bash
# Remove unnecessary outputs and rename generated imagery 
cd /home/jupyter/eie_vision; . scripts/config/flood.sh

rm -r ${root_dir}${temp_dir}${model_name}${temp_ims}/*_input_label.jpg || true # Remove unnecessary outputs; do nothing if files already deleted
rename 's/_pre'${im_suffix}'_synthesized_image.jpg/_post'${im_suffix}'.jpg/' ${root_dir}${temp_dir}${model_name}${temp_ims}*

In [ ]:
!cd /home/jupyter/eie_vision; . scripts/config/flood.sh ;\
cd ${root_dir} ;\
python src/models/Pix2pix-CycleGAN/test.py \
--load_size 1024 --crop_size 1024 --dataroot ${temp_dir}${model_name}${temp_ims} --name . \
--checkpoints_dir ${seg_model_path} \
--model pix2pix --direction AtoB \
--results_dir ${temp_dir}${model_name}${temp_mask} \
--dataset_mode single \
--num_test 3000 --max_dataset_size 2000 --no_flip \
--gpu_ids 0,1,2,3,4,5,6,7 --batch_size 8 # 1

# above code creates segmentation masks in --results-dir 

# --num_test 775
#  --phase to_segment

In [ ]:
!cd /home/jupyter/eie_vision; sh scripts/move_temp_to_results.sh

In [ ]:
# Upload to bucket
cd ../..; . scripts/config/houston_west.sh

split_src=('hold_mask/' 'test_mask/' 'train_mask/')
split_dst=('hold_mask/' 'test_mask/' 'train_mask/')

for ((i=0;i<${#split_src[@]};++i)); do 
    gsutil -m cp -r ${root_dir}${dataroot}${split_src[i]}*.png gs://${bucket_path}${split_dst[i]}
done




### Shutdown instance

In [ ]:
!gcloud compute instances stop --quiet --project "fdl-us-2020-eie" --zone "us-west1-b" eie-week1-ml-pix2pixhd

Stopping instance(s) eie-week1-ml-pix2pixhd...⠛                                